# Assignment
## **1.**

In [14]:
import sqlite3
import requests
from bs4 import BeautifulSoup
from functools import reduce
import pandas as pd
import csv

In [24]:
con=sqlite3.connect('books.db')  # Create a new 'books' database
cur=con.cursor()  
cur.execute('CREATE TABLE books (Book_name TEXT, Price INT, Rate BIT(3), category_name TEXT)')

In [19]:
# Use to DELETE all database conetent
con=sqlite3.connect('books.db')
cur=con.cursor()
cur.execute('DELETE FROM books')
con.commit()
con.close()

## **2.**

In [2]:
url='https://books.toscrape.com/index.html'
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [21]:
lis=soup.find('ul',class_='').find_all('li')
categories=[li.find('a').get_text().strip() for li in lis]
print(categories)

['Travel', 'Mystery', 'Historical Fiction', 'Sequential Art', 'Classics', 'Philosophy', 'Romance', 'Womens Fiction', 'Fiction', 'Childrens', 'Religion', 'Nonfiction', 'Music', 'Default', 'Science Fiction', 'Sports and Games', 'Add a comment', 'Fantasy', 'New Adult', 'Young Adult', 'Science', 'Poetry', 'Paranormal', 'Art', 'Psychology', 'Autobiography', 'Parenting', 'Adult Fiction', 'Humor', 'Horror', 'History', 'Food and Drink', 'Christian Fiction', 'Business', 'Biography', 'Thriller', 'Contemporary', 'Spirituality', 'Academic', 'Self Help', 'Historical', 'Christian', 'Suspense', 'Short Stories', 'Novels', 'Health', 'Politics', 'Cultural', 'Erotica', 'Crime']


In [22]:
domain='https://books.toscrape.com/'
lis=soup.find('ul',class_='').find_all('li')
urls=[domain+li.find('a').get('href') for li in lis]

In [25]:
rates_dic={'Zero':0,'One':1,'Two':2,'Three':3,'Four':4,'Five':5}

con=sqlite3.connect('books.db')
cur=con.cursor()

for url in urls:  #loop on categories
    soup_cat=BeautifulSoup(requests.get(url).text,'html.parser')
    category_name=soup_cat.find('title').get_text().strip().split('|')[0][:-1]
    for article in soup_cat.find_all('article',attrs={'class':'product_pod'}):  #Loop on books
        Rate=rates_dic[article.find('p').get('class')[1]]
        Book_name=article.find('h3').find('a').get('title')
        Price=float(article.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
        cur.execute('INSERT INTO books VALUES (?,?,?,?)',[Book_name,Price,Rate,category_name])
        
    while soup_cat.find('li',attrs={'class':'next'}):  #loop on all pages
        new_page_url=reduce(lambda a,b:a+'/'+b,url.split('/')[:-1])+'/'+soup_cat.find('li',attrs={'class':'next'}).find('a').get('href')
        soup_page=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
        for article in soup_page.find_all('article',attrs={'class':'product_pod'}):  #loop on books
            Rate=rates_dic[article.find('p').get('class')[1]]
            Book_name=article.find('h3').find('a').get('title')
            Price=float(article.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
            cur.execute('INSERT INTO books VALUES (?,?,?,?)',[Book_name,Price,Rate,category_name])
            soup_cat=soup_page    # HTML soup for the working category, to equal, HTML soup for its Next page
            
con.commit()  # Save changes to the database
con.close()

In [31]:
con=sqlite3.connect('books.db')
cur=con.cursor()
cur.execute('SELECT * FROM books')
print(cur.fetchone())
print(cur.fetchone())
con.close()

("It's Only the Himalayas", 45.17, 2, 'Travel ')
('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 49.43, 4, 'Travel ')


In [38]:
# draft
data="Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond"
udata = data.replace(u'\x80\x99s',u'')
udata

'Full Moon over Noahâ Ark: An Odyssey to Mount Ararat and Beyond'

## **4.**

In [17]:
con=sqlite3.connect('books.db')
cur=con.cursor()
headings=[x[1] for x in cur.execute('PRAGMA table_info(books);').fetchall()]

with open('books.csv','w',encoding='utf-8',newline='') as f:
    writer=csv.DictWriter(f,fieldnames=headings)
    writer.writeheader()
    writer=csv.writer(f)
    writer.writerows(cur.execute('SELECT * FROM books WHERE Rate >= 3 OR Price > 20').fetchall())
    
con.close()

In [18]:
df=pd.read_csv('books.csv')
df

,Book_name,Price,Rate,category_name
0,It's Only the Himalayas,45.17,2,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel
2,See America: A Celebration of Our National Par...,48.87,3,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel
4,Under the Tuscan Sun,37.33,3,Travel
...,...,...,...,...
926,new book 1,10.00,3,new cat
927,new book 2,11.00,3,new cat
928,new book 3,12.00,3,new cat
929,new book 4,13.00,3,new cat


In [19]:
df.sample(10)

,Book_name,Price,Rate,category_name
6,The Great Railway Bazaar,30.54,1,Travel
644,Unicorn Tracks,18.78,3,Fantasy
691,The Natural History of Us (The Fine Art of Pre...,45.22,3,Young Adult
754,Olio,23.88,1,Poetry
57,The Guernsey Literary and Potato Peel Pie Society,49.53,1,Historical Fiction
578,Having the Barbarian's Baby (Ice Planet Barbar...,34.96,4,Science Fiction
372,Why Save the Bankers?: And Other Essays on Our...,48.67,2,Nonfiction
760,salt.,46.78,4,Poetry
74,orange: The Complete Collection 1 (orange: The...,48.41,1,Sequential Art
76,"Lumberjanes, Vol. 1: Beware the Kitten Holy (L...",45.61,3,Sequential Art
